# Phân loại bán giám sát 2-View đồng đào tạo (Co-Training)
# Semi-Supervised Learning with Co-Training (2-View)

In [ ]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

from mvlearn.semi_supervised import CTClassifier
from mvlearn.datasets import load_UCImultifeature

Tải Bộ dữ liệu tính năng nhiều chữ số UCI làm ví dụ cho học bán giám sát
Để mô phỏng tình huống học bán giám sát, hãy xóa ngẫu nhiên 98% nhãn.

In [ ]:
# Load multi-view dataset (only 2 classes for simplicity)
data, labels = load_UCImultifeature(select_labeled=[0, 1])
View0, View1 = data[0], data[1]

# Train-test split
View0_train, View0_test, labels_train, labels_test = train_test_split(View0, labels, test_size=0.33, random_state=42)
View1_train, View1_test, labels_train, labels_test = train_test_split(View1, labels, test_size=0.33, random_state=42)

# Randomly remove 98% of the labels
np.random.seed(6)
remove_idx = np.random.rand(len(labels_train)) < 0.98
labels_train[remove_idx] = np.nan
not_removed = np.where(~remove_idx)
print("Remaining labeled sample labels:", labels_train[not_removed])

Đào tạo đồng thời trên 2 chế độ xem so với chế độ xem đơn Học bán giám sát
Ở đây, chúng tôi sử dụng bộ phân loại đồng đào tạo mặc định, sử dụng bộ phân loại Gaussian naive bayes cho cả hai chế độ xem. Chúng tôi so sánh hiệu suất của nó với thiết lập bán giám sát chế độ xem đơn với cùng bộ phân loại cơ bản và với kỹ thuật ngây thơ là ghép nối hai chế độ xem và thực hiện học chế độ xem đơn.

Trong trường hợp này, việc nối hai chế độ xem không cải thiện hiệu suất so với chế độ xem tốt hơn. Đồng đào tạo nhiều chế độ xem vượt trội hơn tất cả.

In [ ]:
# Train GaussianNB on individual views and combined view
gnb0 = GaussianNB()
gnb1 = GaussianNB()
gnb2 = GaussianNB()

gnb0.fit(View0_train[not_removed].squeeze(), labels_train[not_removed])
y_pred0 = gnb0.predict(View0_test)

gnb1.fit(View1_train[not_removed].squeeze(), labels_train[not_removed])
y_pred1 = gnb1.predict(View1_test)

View01_train = np.hstack((View0_train[not_removed].squeeze(), View1_train[not_removed].squeeze()))
View01_test = np.hstack((View0_test, View1_test))
gnb2.fit(View01_train, labels_train[not_removed])
y_pred2 = gnb2.predict(View01_test)

print("Single View Accuracy (View 0):", accuracy_score(labels_test, y_pred0))
print("Single View Accuracy (View 1):", accuracy_score(labels_test, y_pred1))
print("Naive Concatenated View Accuracy:", accuracy_score(labels_test, y_pred2))

In [ ]:
# Co-training with GaussianNB on both views
ctc = CTClassifier()
ctc.fit([View0_train, View1_train], labels_train)
y_pred_ct = ctc.predict([View0_test, View1_test])

print("Co-Training Accuracy (GaussianNB):", accuracy_score(labels_test, y_pred_ct))

OUTPUT:
Độ chính xác của chế độ xem đơn ở chế độ xem đầu tiên: 0,568

Độ chính xác của chế độ xem đơn trên chế độ xem thứ hai: 0,591

Độ chính xác của chế độ xem nối tiếp ngây thơ: 0,591

Độ chính xác của Co-Training trên 2 Lượt xem: 0.992

Chọn các Bộ phân loại cơ sở khác nhau cho các chế độ xem và thay đổi các tham số fit() của CTClassifier
Bây giờ, chúng tôi sử dụng bộ phân loại rừng ngẫu nhiên với các thuộc tính khác nhau cho mỗi chế độ xem. Hơn nữa, chúng tôi chọn thủ công số lượng ví dụ dương (p) và âm (n) được chọn trong mỗi vòng trong quá trình đồng đào tạo và chúng tôi xác định kích thước nhóm không có nhãn để rút chúng ra và số lần lặp lại đào tạo để thực hiện.

Trong trường hợp này, việc nối hai chế độ xem sẽ hiệu quả hơn các phương pháp chế độ xem đơn, nhưng đào tạo đồng thời nhiều chế độ xem vẫn hiệu quả nhất.

In [ ]:
# Use RandomForestClassifier for a new comparison
rfc0 = RandomForestClassifier(n_estimators=100, bootstrap=True)
rfc1 = RandomForestClassifier(n_estimators=6, bootstrap=False)
rfc2 = RandomForestClassifier(n_estimators=100, bootstrap=False)

rfc0.fit(View0_train[not_removed].squeeze(), labels_train[not_removed])
y_pred0 = rfc0.predict(View0_test)

rfc1.fit(View1_train[not_removed].squeeze(), labels_train[not_removed])
y_pred1 = rfc1.predict(View1_test)

rfc2.fit(View01_train, labels_train[not_removed])
y_pred2 = rfc2.predict(View01_test)

print("Single View Accuracy (View 0):", accuracy_score(labels_test, y_pred0))
print("Single View Accuracy (View 1):", accuracy_score(labels_test, y_pred1))
print("Naive Concatenated View Accuracy:", accuracy_score(labels_test, y_pred2))

In [ ]:
# Co-training with RandomForestClassifier and custom parameters
rfc0 = RandomForestClassifier(n_estimators=100, bootstrap=True)
rfc1 = RandomForestClassifier(n_estimators=6, bootstrap=False)
ctc = CTClassifier(rfc0, rfc1, p=2, n=2, unlabeled_pool_size=20, num_iter=100)
ctc.fit([View0_train, View1_train], labels_train)
y_pred_ct = ctc.predict([View0_test, View1_test])

print("Co-Training Accuracy (RandomForest):", accuracy_score(labels_test, y_pred_ct))

Độ chính xác của chế độ xem đơn ở chế độ xem đầu tiên: 0,902

Độ chính xác của chế độ xem đơn trên chế độ xem thứ hai: 0,871

Độ chính xác của chế độ xem nối tiếp ngây thơ: 0,977

Độ chính xác của đào tạo đồng thời: 0,992
Nhận xác suất dự đoán cho tất cả các ví dụ

In [ ]:
# Predict probabilities for the test set
y_pred_proba = ctc.predict_proba([View0_test, View1_test])
print("Shape of predicted probabilities:", y_pred_proba.shape)
print("First 10 predicted class probabilities:")
print(y_pred_proba[:10, :])


Hình dạng y_proba đầy đủ = (132, 2)

Xác suất của 10 lớp đầu tiên:

[[1. 0. ]
 [0,945 0,055 ]
 [0,005 0,995 ]
 [0,09 0,91 ]
 [0,16833333 0,83166667]
 [0,995 0,005 ]
 [0,955 0,045 ]
 [0,955 0,045 ]
 [0,28 0,72 ]
 [0,925 0,075 ]]